In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import os
import cv2
import tensorflow as tf
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization , GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras import optimizers
from keras.optimizers import RMSprop, SGD, Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.callbacks import CSVLogger

In [ ]:
from keras.applications import InceptionResNetV2

row_size = 128
col_size = 128

channels = 3
image_shape = (row_size, col_size, channels)
#image_shape = [128, 128]
num_classes = 6


#downloading base_model
base_model= tf.keras.applications.VGG19(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=image_shape,
    #pooling=None,
    #classes=6,
    #classifier_activation="softmax",
)
#base_model.summary()

In [ ]:
base_model.trainable = False

In [ ]:
# first we need to run the model in infrence mode to remove the effect of batch norm. then we need to fine tune the model.
################################## this is the key. this freezes all the layers in the model one by one.
for layer in base_model.layers:
	layer.trainable = True
##^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
top_model = base_model.output
#top_model= GlobalAveragePooling2D()(top_model)
top_model= Flatten()(top_model)
top_model = Dense(4096,activation='relu')(top_model)
top_model = Dropout(0.4)(top_model)
top_model = Dense(num_classes,activation='softmax')(top_model)

model = Model(inputs=base_model.input, outputs=top_model)

model.summary()


In [ ]:
batch_size = 16
IMAGE_SIZE = [128, 128]

gen = ImageDataGenerator(rescale=1./255,
      #shear_range=0.2,
      #zoom_range=0.2,
      #vertical_flip=True,
      horizontal_flip=True)

train_path = 'F:\Arrhythmia Detection\Images\All_images'
gen = ImageDataGenerator(validation_split=0.2)

train_generator = gen.flow_from_directory(
  train_path,
  target_size=IMAGE_SIZE,
  shuffle=True,
  batch_size=batch_size,
    subset='training' )

valid_generator = gen.flow_from_directory(
  train_path,
  target_size=IMAGE_SIZE,
  shuffle=True,
  batch_size=batch_size,
    subset='validation' )



In [ ]:
filepath = r'F:\Arrhythmia Detection\VGG_arr_detection.h5'
checkpoint = ModelCheckpoint(filepath,
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss',
                              factor = 0.1,
                              patience = 1,
                              verbose = 1,
                              mode='min')
csv_logger = CSVLogger("VGGmodel_history_log.csv",separator=",", append=True)
#csv_logger = CSVLogger('training.log')

In [ ]:
callbacks_list = [checkpoint , csv_logger, reduce_lr]

In [ ]:
opt = optimizers.Adam(lr = 1e-5)
model.compile(loss = 'categorical_crossentropy',
              optimizer = opt,
              metrics = ['accuracy'])

In [ ]:
history = model.fit(
  train_generator,
  validation_data = valid_generator,
  epochs=4,
  steps_per_epoch=79866//batch_size,
  validation_steps=19965//batch_size , callbacks=callbacks_list)

In [ ]:
import matplotlib.pyplot as plt
# Loss Curves
#plt.figure(figsize=[10,8])
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['Training loss', 'Validation Loss'],fontsize=12)
plt.grid()
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Curves',fontsize=16)
plt.savefig("loss_densenet201.png")
plt.show()
 
# Accuracy Curves
#plt.figure(figsize=[8,6])
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=12)
plt.grid()
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Accuracy',fontsize=16)
plt.title('Accuracy Curves',fontsize=16)
plt.savefig("acc_densenet201.png")
plt.show()

In [1]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import sklearn
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
from keras.models import load_model
import pickle 
from keras.preprocessing.image import ImageDataGenerator


path = 'F:/Arrhythmia Detection/Final_2_arr_detection.h5'
#path = 'F:/CA Detection Github/ecg_model_own.hdf5'
model =  load_model(path)
gen = ImageDataGenerator()
IMAGE_SIZE = [128 , 128 ]
batch_size = 32

t_path = 'F:/Arrhythmia Detection/Images/Test'

test_generator = gen.flow_from_directory(
  t_path,
  target_size=IMAGE_SIZE,
  shuffle=False)
  #batch_size=batch_size,
    #subset='validation' )

class_labels = test_generator.class_indices
class_labels = {v: k for k, v in class_labels.items()}
classes = list(class_labels.values())


pred = model.predict(test_generator)
pred = np.argmax(pred,axis = 1) 

print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, pred))
print('Classification Report')
target_names = list(class_labels.values())
print(classification_report(test_generator.classes, pred, target_names=target_names))


CM = confusion_matrix(test_generator.classes, pred)
from mlxtend.plotting import plot_confusion_matrix
fig, ax = plot_confusion_matrix(conf_mat=CM ,  figsize=(12, 8))
plt.xticks(range(6), ["Normal ECG" , "PVC" , "PAB" , "RBB" , "LBB" , "APC"], fontsize=16)
plt.yticks(range(6), ["Normal ECG" , "PVC" , "PAB" , "RBB" , "LBB" , "APC"], fontsize=16)
#plt.savefig("confusion matrix.png")
plt.show()


Found 120 images belonging to 6 classes.


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node model/conv1/conv/Conv2D (defined at <ipython-input-1-1aa5ba5c1e72>:33) ]] [Op:__inference_predict_function_25789]

Function call stack:
predict_function


In [ ]:
from numba import cuda 
device = cuda.get_current_device()
device.reset()

In [ ]:
tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None
)